In [1]:
import requests
import pandas as pd
import datetime as dt
import os
import ast


from bs4 import BeautifulSoup

localpath = r"C:\Users\Admin\Documents\Python Scripts\Flipkart\\"
Base_URL='https://www.flipkart.com/search'

List_File = open(r"C:\Users\Admin\Documents\Python Scripts\Flipkart\Flipkart_Search_List.txt", "r")

bot_token = '2036188374:AAEXsuaS2hRuGfPIhfNm8D9yZqqOEkX8wM8'
bot_chatID = '-559486791'


#looping through all the seaches
for line in List_File:
    line=line.rstrip()
    Search=line.replace(" ","%20")
    tm=str(dt.datetime.now())
    newtm=tm.replace(' ','_').replace(':','_').replace('.','_').replace('-','_')
    LocalFileName="'"+localpath+'\\Laptops_'+newtm+'.txt'+"'"


    Srch_URL=Base_URL+"?q="+Search
    #print(Srch_URL)


    headers={"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}



    page=requests.get(Srch_URL,headers=headers)
    soup=BeautifulSoup(page.content, 'html.parser')
    Title = soup.find_all("div", {"class": "_4rR01T"})
    Title_only=[a.text for a in Title ]
    Price = soup.find_all("div", {"class": "_30jeq3 _1_WHN1"})
    Price_only=[a.text for a in Price ]
    link_only=[]
    for a in soup.find_all("a", {"class": "_1fQZEK"}, href=True):
        full_link='https://www.flipkart.com' + a['href']
        link_only.append(full_link)
    #Totaldict=dict(zip(Title_only,Price_only))
    Pages = soup.find("div", {"class": "_2MImiq"})
    Children=Pages.findChildren("span",recursive=False)
    No_Page=[child for child in Children]
    #print(No_Page[0].text)
    Max_Page=int(No_Page[0].text.rsplit(' ')[3])
    #print(Max_Page)



    i=2
    Final_Title_only=Title_only
    Final_Price_only=Price_only
    Final_Link_only=link_only
    #Finaldict=Search+"Finaldict"
    #print(Finaldict)
    Finaldict={}
    while i <= int(Max_Page):
        counter=str(i)
        Page_Srch_URL=Srch_URL+"&page="+counter
        page1=requests.get(Page_Srch_URL,headers=headers)
        soup=BeautifulSoup(page1.content, 'html.parser')
        Title1 = soup.find_all("div", {"class": "_4rR01T"})
        Title_only1=[a.text for a in Title1 ]
        Final_Title_only=Final_Title_only+Title_only1
        Price1 = soup.find_all("div", {"class": "_30jeq3 _1_WHN1"})
        Price_only1=[a.text for a in Price1 ]
        Final_Price_only=Final_Price_only+Price_only1
        link_only1=[]
        for a in soup.find_all("a", {"class": "_1fQZEK"}, href=True):
            full_link1='https://www.flipkart.com' + a['href']
            link_only1.append(full_link1)
        Final_Link_only=Final_Link_only+link_only1
        i+=1
        Finaldict=dict(zip(Final_Title_only,Final_Price_only))
        hrefdict=dict(zip(Final_Title_only,Final_Link_only))
        #print(hrefdict)

#Read from prev 
    with open(localpath  + Search + '_' + 'prev' + '.txt',encoding="utf-8") as f:
        data = f.read()
        Finaldict1 = ast.literal_eval(data)
        f.close()

#print(Finaldict1)

#write to prev and date file

    with open(localpath  + Search + '_' + newtm + '.txt' ,'w',encoding="utf-8") as data: 
        data.write(str(Finaldict))
        
    with open(localpath  + Search + '_' + 'prev' + '.txt' ,'w',encoding="utf-8") as data: 
        data.write(str(Finaldict))
        

#Comparsion from previous to current
    ResultDict={}
    for item in Finaldict.keys():
        for item1 in Finaldict1.keys():
            if item==item1:
                price1=(Finaldict1[item1].replace('₹','').replace(',',''))
                price2=(Finaldict[item].replace('₹','').replace(',',''))
                diff=int(price1)-int(price2)
                ResultDict[item]=[int(Finaldict1[item1].replace('₹','').replace(',','')),int(Finaldict[item].replace('₹','').replace(',','')),diff]
            
        
        
    outputDict={}
    for item in ResultDict.keys():
        if ResultDict[item][2]!=0:
            perc=(ResultDict[item][2]/ResultDict[item][0])*100
            if perc > 50:
                outputDict[item]=[ResultDict[item][0],perc]
                #print(hrefdict[item])
                send_message= (item +' priced at ₹'+ str(ResultDict[item][0]) + ' is now dropped to ₹' + str(ResultDict[item][1]) +
                     '.....Please click below link to check the product\n' + str(hrefdict[item]))
                #def telegram_bot_sendtext(bot_message):
    
                send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + send_message
                response = requests.get(send_text)
                #return response.json()
                #telegram_bot_sendtext(send_message)